# Validating Variance Process Asymptotic Formula

This notebook validates the theoretical formula for the expected asymptotic variance of a variance process with asymmetric effects.

## Background

We consider a variance process that evolves according to:

$$V_t = \omega + \alpha \cdot \epsilon_{t-1}^2 \cdot V_{t-1} + \gamma \cdot \epsilon_{t-1}^2 \cdot V_{t-1} \cdot I(\epsilon_{t-1} < 0) + \beta \cdot V_{t-1}$$

where:
- $V_t$ is the variance at time $t$
- $\epsilon_t$ are standardized innovations (mean 0, variance 1)
- $\omega, \alpha, \beta, \gamma$ are model parameters
- $I(\epsilon_{t-1} < 0)$ is an indicator function for negative innovations
- The $\gamma$ term captures asymmetric effects (leverage effects)

## Theoretical Result

The expected asymptotic variance is given by:

$$E[V_{\infty}] = \frac{\omega}{1 - \kappa}$$

where $\kappa = \alpha + \beta + \gamma \cdot P_0$ and $P_0 = E[\epsilon^2 \cdot I(\epsilon < 0)]$ is the expected squared innovation conditional on negative innovations.

## Validation Approach

We validate this formula by:
1. Simulating the variance process for multiple paths and time steps
2. Computing the empirical mean of the final variance values
3. Comparing with the theoretical prediction across different:
   - Innovation distributions (normal, uniform, lognormal, biased coin)
   - Parameter combinations
   - Sample sizes

This validates that the theoretical formula correctly captures the long-run expected variance of the process.


In [1]:
import numpy as np
from density_engine.skew_student_t import HansenSkewedT_torch

def make_samples(dist_name, num_paths, num_steps, **kwargs):
    """
    Make samples from a given distribution.
    """
    if dist_name == "normal":
        return np.random.normal(0, 1, (num_paths, num_steps))
    elif dist_name == "uniform":
        return np.random.uniform(-1, 1, (num_paths, num_steps))
    elif dist_name == "lognormal":
        return np.random.lognormal(0, 1, (num_paths, num_steps))
    elif dist_name == "biased_coin":
        return 2*(np.random.uniform(0, 1, (num_paths, num_steps)) < 0.7) - 1
    elif dist_name == "skewed_student":
        eng = HansenSkewedT_torch(eta=10, lam=0.5)
        return eng.rvs((num_paths * num_steps)).cpu().numpy().reshape(num_paths, num_steps)
    else:
        raise ValueError(f"Unknown distribution: {dist_name}")



def make_standard_samples(dist_name, num_paths, num_steps, **kwargs):
    """
    Make standard samples from a given distribution. mean 0, std 1.
    """
    e = make_samples(dist_name, num_paths, num_steps, **kwargs)
    return (e - np.mean(e)) / np.std(e)


def sim_path(*, dist_name, num_paths, num_steps, var0, alpha, beta, gamma, omega, **kwargs):
    """
    Simulate a path of the variance process.
    """
    e = make_standard_samples(dist_name, num_paths, num_steps)
    e2 = e**2

    if dist_name == "skewed_student":
        eng = HansenSkewedT_torch(eta=10, lam=0.5)
        P_0 = eng.second_moment_left().cpu().numpy()
    else:
        P_0 = np.mean(e*e * (e < 0))

    var = np.zeros((num_paths, num_steps))
    var[:, 0] = var0
    for t in range(1, num_steps):
        eps2 = e2[:, t-1] * var[:, t-1]
        var[:, t] = omega + alpha*eps2 + gamma*eps2*P_0 + beta*var[:, t-1]
            
    return var

def theoretical_var_inf(*, alpha, beta, gamma, omega, dist_name, **kwargs):
    """
    Calculate the theoretical expected variance of the variance process.
    """
    z = make_standard_samples(dist_name, 200000, 1).ravel()
    P_0 = np.mean(z*z * (z < 0))
    kappa = alpha + beta + gamma * P_0
    return omega / (1 - kappa), P_0


# Settings for the experiments.
cases = [
    {
        "num_paths": 100_000,
        "num_steps": 1_000,
        "alpha": 0.15,
        "beta": 0.1,
        "gamma": 0.3,
        "omega": 0.5,
        "var0": 0.33,
        "dist_name": "skewed_student"
    },     
    {
        "num_paths": 100_000,
        "num_steps": 1_000,
        "alpha": 0.2,
        "beta": 0.4,
        "gamma": 0.1,
        "omega": 0.25,
        "var0": 0.33,
        "dist_name": "biased_coin"
    },    
    {
        "num_paths": 100_000,
        "num_steps": 1000,
        "alpha": 0.1,
        "beta": 0.2,
        "gamma": 0.2,
        "omega": 3.14,
        "var0": 0.1,
        "dist_name": "normal"
    },
    {
        "num_paths": 100_000,
        "num_steps": 1_000,
        "alpha": 0.4,
        "beta": 0.1,
        "gamma": 0.1,
        "omega": 1.3,
        "var0": 0.4,
        "dist_name": "lognormal"
    },      

]

# Run the experiments.
for settings in cases:
    theo_var_inf, P_0 = theoretical_var_inf(**settings)
    print(f'\ndistribution = {settings["dist_name"]}, P_0 = {P_0:.2f}, alpha = {settings["alpha"]}, beta = {settings["beta"]}, gamma = {settings["gamma"]}, omega = {settings["omega"]}')

    # repeat experiment 5 times:
    for _ in range(5):
        # Simulation based
        var = sim_path(**settings)
        exp_var_inf = np.mean(var[:, -1])
        print(f"Simulated expected variance: {exp_var_inf:.2f}, theoretical: {theo_var_inf:.2f}")



distribution = skewed_student, P_0 = 0.37, alpha = 0.15, beta = 0.1, gamma = 0.3, omega = 0.5
Simulated expected variance: 0.78, theoretical: 0.78
Simulated expected variance: 0.78, theoretical: 0.78
Simulated expected variance: 0.78, theoretical: 0.78
Simulated expected variance: 0.78, theoretical: 0.78
Simulated expected variance: 0.79, theoretical: 0.78

distribution = biased_coin, P_0 = 0.70, alpha = 0.2, beta = 0.4, gamma = 0.1, omega = 0.25
Simulated expected variance: 0.76, theoretical: 0.76
Simulated expected variance: 0.76, theoretical: 0.76
Simulated expected variance: 0.76, theoretical: 0.76
Simulated expected variance: 0.76, theoretical: 0.76
Simulated expected variance: 0.76, theoretical: 0.76

distribution = normal, P_0 = 0.50, alpha = 0.1, beta = 0.2, gamma = 0.2, omega = 3.14
Simulated expected variance: 5.25, theoretical: 5.23
Simulated expected variance: 5.24, theoretical: 5.23
Simulated expected variance: 5.24, theoretical: 5.23
Simulated expected variance: 5.24, th